# Projeto Aprendizado de Máquina

## CMC-13 Introdução a Ciência de Dados
## Equipe:
### André Luiz de Melo Thiessen
### Nikollas da Silva Antes
### Pedro Anacleto Martins Senna de Oliveira

In [55]:
#Importando as bibliotecas necessárias
import pandas as pd
from datetime import datetime
from datetime import date

#Importando as bases de dados
movies = pd.read_csv("movies.csv", on_bad_lines='skip', sep = ';')
ratings = pd.read_csv("ratings.csv", sep = ';')
users = pd.read_csv("users.csv")

# Verificando a existência de valores nulos (dados faltantes)
print(movies.isnull().values.any())
print(ratings.isnull().values.any())
print(users.isnull().values.any())

#Apagando as colunas irrelevantes
movies = movies.drop(columns='Title')
ratings = ratings.drop(columns='Timestamp')
users = users.drop(columns='Zip-code')
users = users.drop(columns='name')

False
False
False


In [56]:
###MOSTRAR AS TABELAS
display(movies)
display(ratings)
display(users)

,MovieID,Genres
0,1,Animation|Children's|Comedy
1,2,Adventure|Children's|Fantasy
2,3,Comedy|Romance
3,4,Comedy|Drama
4,5,Comedy
...,...,...
3877,3948,Comedy
3878,3949,Drama
3879,3950,Drama
3880,3951,Drama


,UserID,MovieID,Rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


,UserID,Gender,Occupation,birthday
0,1,F,10,4/1/2012
1,2,M,16,9/21/1964
2,3,M,15,4/3/1995
3,4,M,7,5/8/1974
4,5,M,20,6/18/1996
...,...,...,...,...
6035,6036,F,15,10/17/1995
6036,6037,F,1,6/12/1975
6037,6038,F,1,1/17/1963
6038,6039,F,0,10/6/1977


In [57]:
format_string = "%m/%d/%Y"
def calculateAge(birthDate): 
    today = date.today() 
    age = today.year - birthDate.year - ((today.month, today.day) < (birthDate.month, birthDate.day))
    return age

for i in users.index:
    [month, day, year] = users['birthday'][i].split('/')
    if(day == '0'):
        users.loc[i, 'birthday'] = month+'/'+'1/'+year
    date = datetime.strptime(users['birthday'][i], format_string)
    age = calculateAge(date)
    users.loc[i, 'birthday'] = age


In [58]:
#Agrupando os usuários em 
for i in users.index:
    if(int(users['birthday'][i])<18):
        users.loc[i, 'birthday'] = '1'
    elif (int(users['birthday'][i])<24):
        users.loc[i, 'birthday'] = '18'
    elif (int(users['birthday'][i])<35):
        users.loc[i, 'birthday'] = '25'
    elif (int(users['birthday'][i])<45):
        users.loc[i, 'birthday'] = '35'
    elif (int(users['birthday'][i])<50):
        users.loc[i, 'birthday'] = '45'
    elif (int(users['birthday'][i])<55):
        users.loc[i, 'birthday'] = '50'
    else:
        users.loc[i, 'birthday'] = '56'
        

In [59]:
display(users)

,UserID,Gender,Occupation,birthday
0,1,F,10,1
1,2,M,16,56
2,3,M,15,25
3,4,M,7,45
4,5,M,20,25
...,...,...,...,...
6035,6036,F,15,25
6036,6037,F,1,45
6037,6038,F,1,56
6038,6039,F,0,35
